# About

Read in stock values.

In [20]:
from os.path import join
from os import listdir
import random
import datetime as dt

import pandas as pd
import numpy as np

## Load data

In [23]:
SOURCE_DATA_FOLDER = ['..', 'datasets']
STOCK_DATA_FOLDER = join(*SOURCE_DATA_FOLDER,'kaggle','paultimothymooney','nasdaq')
MONTHS = ('12-2019','01-2020','02-2020','03-2020')

stocks = []
for stock_file in listdir(STOCK_DATA_FOLDER):
    try:
        stock_ = pd.read_csv(join(STOCK_DATA_FOLDER,stock_file))
        stock_ = stock_[stock_['Date'].str.endswith(MONTHS)]
        l_date = list(stock_['Date'])
        l_close = list(stock_['Close'])

        for i in range(len(l_date)):
            d = l_date[i]
            l_date[i] = f'{d[6:]}-{d[3:5]}-{d[:2]}'
        stock_rows = list(zip(l_date,l_close))
        stock_ = pd.DataFrame(stock_rows, columns=['Date','Value'])
        stock_.sort_values(by='Date', inplace=True)
        stocks.append(stock_)
    except:
        print('Error reading',stock_file)
print('Stock count:',len(stocks))

Stock count: 19


### Fill missing days

In [40]:
# inverted month filter
MONTHS_i = ('2020-01','2020-02','2020-03')
# end date treshold
end_date = dt.date(2020,4,1)
# pad from here
begin_date = dt.date(2019,12,1)
one_day = dt.timedelta(1) # delta to add in loop
stocks_mod = []
missing = 0
# fix all stocks
for stock in stocks:
    # begin date
    date = begin_date
    # put dates in dictionary for easy lookup
    dates = {}
    for i in range(len(stock)):
        dates[stock['Date'].iloc[i]] = stock['Value'].iloc[i]
    # last stockl value used for padding missing value
    last_val = None
    dates_to_add = []
    values_to_add = []
    # chec kal ldated in interval
    while date < end_date:
        # get date str
        curr = date.isoformat()
        # if not in dates, pad (but only if we god pad value)
        if not curr in dates and last_val:
            dates_to_add.append(curr)
            values_to_add.append(last_val)
        elif curr in dates:
            # get next pad value if we ghave one
            last_val = dates[curr]
        # add a day
        date = date + one_day
    # merge paddings if we found gaps we filled
    if dates_to_add != []:
        missing += 1
        c_date = list(stock['Date'])+dates_to_add
        c_value = list(stock['Value'])+values_to_add
        s_ = pd.DataFrame(zip(c_date,c_value), columns=['Date','Value'])
        # finter out unwanted dates
        s_ = s_[s_['Date'].str.startswith(MONTHS_i)]
        s_.sort_values(by='Date',inplace=True)
        stocks_mod.append(s_)
    else:
        stocks_mod.append(stock)
# replace
stocks = stocks_mod
if missing>0:
    print(f'Fixed {missing} stocks.')

### Output

In [42]:
i=0
stocks_mod = []
for stock in stocks:
    label = chr(ord('A')+i)
    stocks_mod.append(stock.eval('Label = @label'))
    i+=1
df_out = pd.concat(stocks_mod)
print('Writing DataFrame to file.')
df_out.to_csv(join(*SOURCE_DATA_FOLDER,'stock_data.csv'))
print('Bye.')

Writing DataFrame to file.
Bye.
